In [26]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
read = pd.read_csv('tweet_emotions.csv')
read.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [29]:
def data_processing(content):
    content = content.lower()
    content = re.sub(r'\@','', content)
    content_token = word_tokenize(content)
    filtered_content = [w for w in content_token if not w in stop_words]
    return " ".join(filtered_content)

In [30]:
read.content = read['content'].apply(data_processing)

In [31]:
read.head()

,tweet_id,sentiment,content
0,1956967341,empty,tiffanylue know listenin bad habit earlier sta...
1,1956967666,sadness,layin n bed headache ughhhh ... waitin call ...
2,1956967696,sadness,funeral ceremony ... gloomy friday ...
3,1956967789,enthusiasm,wants hang friends soon !
4,1956968416,neutral,dannycastillo want trade someone houston ticke...


In [32]:
vect = TfidfVectorizer(ngram_range=(1,2)).fit(read['content'])
feature_names = vect.get_feature_names()
print(feature_names[:20])

['00', '00 03', '00 88', '00 already', '00 amp', '00 bill', '00 bottle', '00 car', '00 curtain', '00 end', '00 est', '00 feel', '00 god', '00 graduated', '00 http', '00 injured', '00 ins', '00 kickoff', '00 like', '00 ll']


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
